<a href="https://colab.research.google.com/github/btcain44/Applied_Deep_Learning/blob/main/Enhanced_Transfer_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Bi-Weekly Report #5
### Brian Cain
#### Enhanced_Transfer_Learning.ipynb

In my previous attempts I have had lackluster performance in attempting to classify the Cifar-100 dataset. The best results I got were in my Bi-Weekly Report #3 when I made a Generalist-Specialist model with 39.62% accuracy. 

I now will try to extend the lessons learned in the <b>Comparitive_Transfer_Learning.ipynb</b> notebook to apply transfer learning to the CIFAR-100 classification problem. Here are a couple of lessons learned about transfer learning in that notebook:
* Add layers after ImageNet weights to give the network more ability to learn about Task B
* Turn on Fine Tuning of Task A's weights when training Task B so the network can adjust the ImageNet weights to boost classification 

Now I will import the CIFAR-100 dataset. 

In [1]:
##Import necessary packages
import numpy as np
import tensorflow as tf

##Load the CIFAR-100 dataset
from tensorflow import keras
(x_train, y_train), (x_test, y_test) = keras.datasets.cifar100.load_data()
assert x_train.shape == (50000, 32, 32, 3)
assert x_test.shape == (10000, 32, 32, 3)
assert y_train.shape == (50000, 1)
assert y_test.shape == (10000, 1)
 

##Format the labels so we have shape (50000,) and (10000,) respectively 
y_train = np.array([i[0] for i in y_train])
y_test = np.array([i[0] for i in y_test])


169017344/169001437 [==============================] - 6s 0us/step


### Transfer Learning CIFAR-100 by Going Deeper

Lets craft up the enhanced architecture of the model with Fine-Tuning enabled. Here are some additional features of this architecture and what they are aimed at solving:
* <b>Fine-Tuning ImageNet Weights:</b> The results in Comparitive_Transfer_Learning.ipynb made it obvious that transfer learning from ResNet50 with additional dense layers could only do so much. By Fine-tuning weights we can improve accuracy by learning more about the data in our specific classification problem rather than just relying on static weights designed for ImageNet
* <b>Add Convolutions After ResNet50 ImageNet Weights:</b> Another hypothesized downfall of the transfer learning architecture used in Comparitive_Transfer_Learning.ipynb was the fact that only a single dense layer was used after the ImageNet weights to output a prediction. This time around, I made a deeper network by adding two 1x1 convolutions combined with Global Average Pooling to give the Cifar-100 network a change to learn even more about the data. However, I am concerned if over-fitting may occur because of this. 
* <b>Adam Optimizer with Learning Rate .001:</b> In previous attempts to classify the CIFAR-100 dataset, I have achieved extremely low accuracy. This time, I chose to specify a relatively low learning rate for the Adam optimizer so that the model will be less likely to miss optimal weights when Fine-Tuning the network and creating weights for new convolutions. This will add compute time but I am hoping it is worth it. 

In [2]:
##Import the ResNet50 Architecture
import tensorflow as tf
from tensorflow.keras.applications import ResNet50

#Obtain Imagenet model weights
feature_extractor = ResNet50(weights='imagenet', 
                             input_shape=(32, 32, 3),
                             include_top=False)

#For better performance, make it so that we are Fine-Tuning imagenet weights during transfer learning
feature_extractor.trainable = True

#Define dimensions for the
input_ = tf.keras.Input(shape=(32, 32, 3))

#Create layer that extracts features
x = feature_extractor(input_, training=True)

##Add additional convolutions to see if this helps network become more accuracte
x = tf.keras.layers.Conv2D(1024, (1, 1), activation='relu', input_shape=(1,1,2048))(x)
x = tf.keras.layers.Conv2D(612, (1, 1), activation='relu', input_shape=(1,1,1024))(x)

#Perform global average pooling to condense ResNet50 output
x = tf.keras.layers.GlobalAveragePooling2D()(x)

# Set the final layer with sigmoid activation function
x = tf.keras.layers.Dense(306, activation='relu')(x)
output_ = tf.keras.layers.Dense(100, activation='softmax')(x)

#Make an instance of the transfer learning architecture
cifar_100_deep = tf.keras.Model(input_, output_)

#Compile the model
cifar_100_deep.compile(optimizer=tf.keras.optimizers.Adam(lr=0.001),
                             loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
                             metrics=['accuracy'])

94781440/94765736 [==============================] - 1s 0us/step


/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/optimizer_v2.py:356: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


Now let's split the train/validation set and train the model:

In [3]:
##Conduct a step of data normalization
x_train, x_test = x_train / 255.0, x_test / 255.0

##Perform train/validation split on the data
from sklearn.model_selection import train_test_split
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.25, random_state=42)

In [8]:
##Fit and train the model
cifar_deep_fit = cifar_100_deep.fit(x_train, y_train, batch_size=100, epochs=50, 
                                   validation_data=(x_val, y_val))

Epoch 1/50
375/375 [==============================] - 58s 155ms/step - loss: 2.8809 - accuracy: 0.2619 - val_loss: 2.7573 - val_accuracy: 0.3009
Epoch 2/50
375/375 [==============================] - 57s 152ms/step - loss: 2.4880 - accuracy: 0.3509 - val_loss: 2.5206 - val_accuracy: 0.3564
Epoch 3/50
375/375 [==============================] - 57s 152ms/step - loss: 2.2314 - accuracy: 0.4077 - val_loss: 2.4476 - val_accuracy: 0.3681
Epoch 4/50
375/375 [==============================] - 56s 151ms/step - loss: 2.0522 - accuracy: 0.4471 - val_loss: 2.2999 - val_accuracy: 0.4159
Epoch 5/50
375/375 [==============================] - 56s 150ms/step - loss: 1.7898 - accuracy: 0.5054 - val_loss: 2.3593 - val_accuracy: 0.4024
Epoch 6/50
375/375 [==============================] - 57s 152ms/step - loss: 1.6539 - accuracy: 0.5390 - val_loss: 2.2957 - val_accuracy: 0.4335
Epoch 7/50
375/375 [==============================] - 57s 151ms/step - loss: 1.4620 - accuracy: 0.5857 - val_loss: 2.2912 - val_ac

Now lets Assess the Test Results:

In [9]:
##Evaluate the testing accuracy of the model
print('Test Accuracy of CIFAR-100 Transfer Learned:')
cifar_100_deep.evaluate(x_test,  y_test, verbose=0)[1]

Test Accuracy of CIFAR-100 Transfer Learned:


0.44609999656677246

To start off with a positive, this transfer learning technique did surpass my previous best accuray on CIFAR-100 of 39.62% with a test accuracy of 44.61%. 

However, there ended up being a crazy amount of over-fitting present in this CIFAR-100 classification model using ResNet50 ImageNet transfer learning. I'm not sure exactly what would account for that to happen but perhaps the addition of the 1x1 convolutional layers and a lower learning rate had something to do with it. 

### Transfer Learning CIFAR-100 going Less Deep and More Data

One more experiment I would like to try with transfer learning to see if I can improve CIFAR-100 classification is to make the network slightly less deep and give it more training data. Here are the ways I will do so:
* <b>Get rid of additional convolutional layers after ImageNet weights</b>
* <b>Mixup Data Augmentation:</b> In Bi-Weekly report 2, I attempted using Mixup Data Augmentation on the CIFAR-100 dataset. Although the results weren't great, it did prevent over-fitting. I will re-train the network here using Mixup Augmentation and hope to see less over-fitting. 

Define function for mixup data augmentation:

In [10]:
##Define a function that performs mixup two images 
def mixup(image1, image2, label1, label2, beta_params):
    
    ##Generate sample from lambda distribution 
    lambda_val = np.random.beta(beta_params[0], beta_params[1])
    
    ##Perform mix-up operation 
    newImg = lambda_val*image1 + (1-lambda_val)*image2
    newLabel = round(lambda_val*label1 + (1-lambda_val)*label2)
    
    return newImg.astype(int), newLabel

Now perform mix-up on a segment of the training data:

In [11]:
##Alter a random 20% of the training data 
np.random.seed(0)
rand_images = np.random.randint(1, 37500, size=int(37500*.2), dtype=int)

##Create training dataset using mixup
x_train_mixup = []
y_train_mixup = []
for i in rand_images:
    mixup_result = (mixup(x_train[i],x_train[i-1],y_train[i],y_train[i-i],[.2,.2]))
    x_train_mixup.append(mixup_result[0])
    y_train_mixup.append(mixup_result[1])

##Translate training data into numpy arrays
x_train_mixup = np.array(x_train_mixup)
y_train_mixup = np.array(y_train_mixup)

##Concatenate new data onto existing training data
x_train_mixup = np.concatenate((x_train, x_train_mixup), axis=0)
y_train_mixup = np.concatenate((y_train, y_train_mixup),axis=None)

Now we will make the alterations to the model described above:

In [20]:
#Obtain Imagenet model weights
feature_extractor = ResNet50(weights='imagenet', 
                             input_shape=(32, 32, 3),
                             include_top=False)

#For better performance, make it so that we are Fine-Tuning imagenet weights during transfer learning
feature_extractor.trainable = True

#Define dimensions for the
input_ = tf.keras.Input(shape=(32, 32, 3))

#Create layer that extracts features
x = feature_extractor(input_, training=True)

#Perform global average pooling to condense ResNet50 output
x = tf.keras.layers.GlobalAveragePooling2D()(x)

# Set the final layer with sigmoid activation function
output_ = tf.keras.layers.Dense(100, activation='softmax')(x)

#Make an instance of the transfer learning architecture
cifar_100_shallow = tf.keras.Model(input_, output_)

#Compile the model
cifar_100_shallow.compile(optimizer='Adam',
                             loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
                             metrics=['accuracy'])

Also it is important to note that in this model I had to use a larger batch size of 200 over 25 epochs to speed up training because Google had previously interrupted my session for going over the GPU limits so I wanted to make sure I could fit this training session in without being interrupted.  

In [23]:
##Fit and train the model
cifar_shallow_fit = cifar_100_shallow.fit(x_train_mixup, y_train_mixup, batch_size=200, epochs=25, 
                                   validation_data=(x_val, y_val))

Epoch 1/25
225/225 [==============================] - 101s 218ms/step - loss: 3.8810 - accuracy: 0.1421 - val_loss: 3.9590 - val_accuracy: 0.1653
Epoch 2/25
225/225 [==============================] - 48s 212ms/step - loss: 3.3430 - accuracy: 0.2317 - val_loss: 3.3893 - val_accuracy: 0.2366
Epoch 3/25
225/225 [==============================] - 47s 211ms/step - loss: 2.7791 - accuracy: 0.3279 - val_loss: 3.1196 - val_accuracy: 0.3070
Epoch 4/25
225/225 [==============================] - 48s 212ms/step - loss: 2.3928 - accuracy: 0.4073 - val_loss: 2.8685 - val_accuracy: 0.3496
Epoch 5/25
225/225 [==============================] - 48s 212ms/step - loss: 2.2185 - accuracy: 0.4477 - val_loss: 2.8156 - val_accuracy: 0.3532
Epoch 6/25
225/225 [==============================] - 48s 212ms/step - loss: 1.9735 - accuracy: 0.5015 - val_loss: 2.5080 - val_accuracy: 0.3919
Epoch 7/25
225/225 [==============================] - 48s 211ms/step - loss: 1.6819 - accuracy: 0.5713 - val_loss: 2.5389 - val_a

In [24]:
##Evaluate the testing accuracy of the model
print('Test Accuracy of CIFAR-100 Transfer Learned with Data Augmentation and Shallower Framework:')
cifar_100_shallow.evaluate(x_test,  y_test, verbose=0)[1]

Test Accuracy of CIFAR-100 Transfer Learned with Data Augmentation and Shallower Framework:


0.38199999928474426

It might be unfair to directly compare this shallower transfer learning model to the one above because of increased batch size and decreased epochs, but it seems that my attempts to decrease over-fitting and increase accuracy have actually made the model worse. The test accuracy for this model is 38.2%, which is a decrease from the Generalist Specialist model made in report #3. So, it seems that by removing those convolutions I had added in the deeper transfer learning model, that we ended up losing some predictive power. I don't think that any predictive power was lost by adding the data augmentation because typically data augmentation is empirically shown to boost performance. 

## Final Thoughts

The CIFAR-100 classification problem remains a tough one to solve, at least for me. However, using Transfer Learning with ImageNet weights from ResNet50 broke my previous record for best accuracy attained on the CIFAR-100 dataset. My previous record was obtained through a laborious process of making Generalist and Specialist models for coarse and fine labels in CIFAR-100. Transfer Learning required significantly less modeling and better results. It would be interesting in the future to try and potentially combine Generalist Specialist and Transfer learning frameworks to see if that works any better. Through the experiments in this notebook and in Comparitive_Transfer_Learning.ipynb, it is easy to see why Transfer Learning has become so relevant as it not only boosts performance but also efficiently recycles large models to create new accurate models with less work. 